In [1]:
import pandas as pd
import math

In [2]:
player_time = pd.read_csv('player_time.csv')
player_ratings = pd.read_csv('player_ratings.csv')

In [3]:
player_time.head()


,match_id,times,gold_t_0,lh_t_0,xp_t_0,gold_t_1,lh_t_1,xp_t_1,gold_t_2,lh_t_2,...,xp_t_129,gold_t_130,lh_t_130,xp_t_130,gold_t_131,lh_t_131,xp_t_131,gold_t_132,lh_t_132,xp_t_132
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,60,409,0,63,142,1,186,168,0,...,62,345,6,351,100,0,77,613,1,125
2,0,120,546,0,283,622,4,645,330,0,...,543,684,12,805,200,0,210,815,5,323
3,0,180,683,1,314,927,9,1202,430,0,...,842,958,16,1135,300,0,210,1290,8,527
4,0,240,956,1,485,1264,11,1583,530,0,...,1048,1500,26,1842,400,0,210,1431,9,589


In [4]:
player_ratings.head()

,account_id,total_wins,total_matches,trueskill_mu,trueskill_sigma
0,236579,14,24,27.868035,5.212361
1,-343,1,1,26.544163,8.065475
2,-1217,1,1,26.521103,8.114989
3,-1227,1,1,27.248025,8.092217
4,-1284,0,1,22.931016,8.092224


In [5]:
players = pd.read_csv('players.csv')

In [6]:
players.head()

,match_id,account_id,hero_id,player_slot,gold,gold_spent,gold_per_min,xp_per_min,kills,deaths,...,unit_order_glyph,unit_order_eject_item_from_stash,unit_order_cast_rune,unit_order_ping_ability,unit_order_move_to_direction,unit_order_patrol,unit_order_vector_target_position,unit_order_radar,unit_order_set_item_combine_lock,unit_order_continue
0,0,0,86,0,3261,10960,347,362,9,3,...,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,51,1,2954,17760,494,659,13,3,...,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,83,2,110,12195,350,385,0,4,...,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2,11,3,1179,22505,599,605,8,4,...,1.0,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,3,67,4,3307,23825,613,762,20,3,...,3.0,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
teamfights = pd.read_csv('teamfights.csv')
teamfights.head()

,match_id,start,end,last_death,deaths
0,0,220,252,237,3
1,0,429,475,460,3
2,0,900,936,921,3
3,0,1284,1328,1313,3
4,0,1614,1666,1651,5


In [8]:
teamfights_players = pd.read_csv('teamfights_players.csv')

To predict the kill-death ratio for a team fight, we will need to first establish the class variable by melting the dataframe so each row represents a teamfight for one team (two rows per fight). 
Before modifying the teamfight_players dataset to form the class variable we will need to extract some other pieces of information first, specifically xp delta and a column to estimate whether the hero was in the fight or not based on gold and xp delta. 

In [9]:
teamfights_players = teamfights_players[0:1000]#cutting data to test feasibility first.

In [10]:
#remove damage, buyback columns
teamfights_players = teamfights_players.drop(['damage', 'buybacks'], axis = 1)

In [11]:
#add 'active' column
def isActive(gold, xp, slot):#A player is 'active' during (not necessarily present in) a team fight if the gain gold and xp
    if gold==0 and xp==0:
        return -1
    else:
        return slot
    
teamfights_players['active'] = teamfights_players.apply(
    lambda x: isActive(x['xp_end'] - x['xp_start'], x['gold_delta'], x['player_slot']), axis = 1)

In [12]:
#unpivot match_id, player_slot, fight_id as index with sum of deaths and participating player slot matrix
#add fight index
teamfights_players['fight_id'] = pd.Series(teamfights_players.index.values/10).apply(math.floor)

In [13]:
teamfights_players.head()

,match_id,player_slot,deaths,gold_delta,xp_end,xp_start,active,fight_id
0,0,0,0,173,536,314,0,0
1,0,1,1,0,1583,1418,1,0
2,0,2,0,0,391,391,-1,0
3,0,3,0,123,1775,1419,3,0
4,0,4,0,336,1267,983,4,0


In [14]:
#calculate Kill-Death ratio
#sum deaths for each team, for each fight
#add team index
def findTeam(player_slot):
    if player_slot < 100:
        return True
    else:
        return False
KDR = teamfights_players[['match_id', 'player_slot', 'deaths', 'fight_id']]
KDR['radiant'] = KDR.apply(lambda x: findTeam(x['player_slot']), axis = 1)
KDR.head()

C:\Users\R\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,match_id,player_slot,deaths,fight_id,radiant
0,0,0,0,0,True
1,0,1,1,0,True
2,0,2,0,0,True
3,0,3,0,0,True
4,0,4,0,0,True


In [15]:
deathCount = KDR.groupby(['fight_id', 'radiant'])['deaths'].sum()


In [16]:
deathCount[0:20]

fight_id  radiant
0         False      2
          True       1
1         False      3
          True       0
2         False      3
          True       0
3         False      3
          True       0
4         False      3
          True       2
5         False      0
          True       3
6         False      4
          True       2
7         False      3
          True       2
8         False      3
          True       2
9         False      3
          True       1
Name: deaths, dtype: int64

In [17]:
#create table with columns for each player slot
active = teamfights_players.pivot_table(columns = ['player_slot'], index = ['match_id', 'fight_id'])
active = active['active']
active.head()

player_slot        0    1    2    3    4    128  129  130  131  132
match_id fight_id                                                  
0        0           0    1   -1    3    4  128  129  130   -1  132
         1           0    1    2    3    4  128  129  130  131  132
         2           0    1    2    3    4  128   -1  130  131  132
         3           0    1    2    3    4  128  129  130  131  132
         4           0    1    2    3    4  128  129  130  131  132

In [18]:
#take the players in a fight, the match, time and desited metric 
#return the sum of that metric for players active in the fight
def pre_stats(time, match, players, metric):
    #measures = ['gold', 'xp', 'lh']
    stats = player_time.loc[((time - 60) < player_time['times']) & (player_time['times'] <= time) & (player_time['match_id'] == match)]

    team = sum([stats[metric + '_t_' + str(players[i])] for i in range(5) if players[i] >= 0]).iloc[0]/sum(1 for i in range(5) if players[i] >= 0)
    
    return team

In [19]:
#Create a dataframe for cumulative team stats
#get a dataframe of fight_id, time, and player indicies
#teamfights = teamfights.reset_index()
activeF = active.reset_index()
playerTimes = teamfights.merge(activeF, left_index = True, right_index = True)
#Add aggregate stat columns
aggStats = playerTimes[['fight_id']]#dataframe to store aggregates
playerTimes = playerTimes.rename(index = str, columns = {0:'P0', #nothing likes integer named columns 
                                            1:'P1', 
                                            2:'P2', 
                                            3:'P3', 
                                            4:'P4', 
                                            128:'P128', 
                                            129:'P129', 
                                            130:'P130', 
                                            131:'P131', 
                                            132:'P132', 
                                            133:'P133',
                                            'match_id_x':'match_id'})
playerTimes
#gold
aggStats['radiant_gold'] = playerTimes.apply(
    lambda x: pre_stats(x['start'], x['match_id'], [x['P0'], x['P1'], x['P2'], x['P3'], x['P4']], 'gold'), axis = 1).values
aggStats['dire_gold'] = playerTimes.apply(
    lambda x: pre_stats(x['start'], x['match_id'], [x['P128'], x['P129'], x['P130'], x['P131'], x['P132']], 'gold'), axis = 1).values
aggStats['radiant_xp'] = playerTimes.apply(
    lambda x: pre_stats(x['start'], x['match_id'], [x['P0'], x['P1'], x['P2'], x['P3'], x['P4']], 'xp'), axis = 1).values
aggStats['dire_xp'] = playerTimes.apply(
    lambda x: pre_stats(x['start'], x['match_id'], [x['P128'], x['P129'], x['P130'], x['P131'], x['P132']], 'xp'), axis = 1).values
aggStats['radiant_lh'] = playerTimes.apply(
    lambda x: pre_stats(x['start'], x['match_id'], [x['P0'], x['P1'], x['P2'], x['P3'], x['P4']], 'lh'), axis = 1).values
aggStats['dire_lh'] = playerTimes.apply(
    lambda x: pre_stats(x['start'], x['match_id'], [x['P128'], x['P129'], x['P130'], x['P131'], x['P132']], 'lh'), axis = 1).values

In [20]:
aggStats[0:10]

,fight_id,radiant_gold,dire_gold,radiant_xp,dire_xp,radiant_lh,dire_lh
0,0,765.000000,939.250000,814.500000,833.250000,5.75,10.000000
1,1,1708.800000,1953.200000,1812.000000,1789.000000,15.40,21.600000
2,2,4662.200000,4717.000000,4805.000000,4578.750000,48.60,58.750000
3,3,7314.000000,6465.200000,7821.200000,6307.600000,72.80,78.600000
4,4,9525.400000,8266.400000,10450.600000,8163.200000,94.00,93.200000
5,5,9078.333333,6371.333333,9220.333333,7100.666667,71.00,67.333333
6,6,10663.000000,9362.200000,11727.400000,9411.800000,102.40,98.600000
7,7,12029.200000,10603.600000,13767.200000,11302.800000,105.60,100.600000
8,8,12547.200000,10268.000000,14758.800000,10925.250000,107.20,83.250000
9,9,14659.000000,12656.000000,17473.200000,13403.250000,126.00,113.750000


Once this is done, the dataset can be grouped to show one row per team per teamfight. The columns would include the total kills, total XP, and one one column per player slot indicating whether they were present in the fight or not based on their XP and gold delta. 

In [21]:
player_matches = players[['match_id', 'account_id', 'player_slot']]
player_matches.head()

,match_id,account_id,player_slot
0,0,0,0
1,0,1,1
2,0,0,2
3,0,2,3
4,0,3,4


In [22]:
player_ratings.set_index('account_id')
player_ratings.head()

,account_id,total_wins,total_matches,trueskill_mu,trueskill_sigma
0,236579,14,24,27.868035,5.212361
1,-343,1,1,26.544163,8.065475
2,-1217,1,1,26.521103,8.114989
3,-1227,1,1,27.248025,8.092217
4,-1284,0,1,22.931016,8.092224


In [23]:
match_ratings = pd.merge(player_matches, player_ratings, how='left', on='account_id', sort=False)
#Aanonomous players will wash out the wins and matches data. 
#These players shoulde not be removed as they still participate in fights and a lot of data would be missing without them.
#For now, I will find the average number of wins and matches for identifiable players, 
#and assign this value to all '0' account_ids
#remove player 0
avgGame = match_ratings[['account_id', 'total_wins', 'total_matches']]
avgGame = avgGame.loc[avgGame['account_id'] > 0]
win = avgGame['total_wins'].mean()
match = avgGame['total_matches'].mean()

match_ratings.loc[match_ratings['account_id'] == 0,'total_wins'] = win
match_ratings.loc[match_ratings['account_id'] == 0,'total_matches'] = match

In [24]:
match_ratings.head()

,match_id,account_id,player_slot,total_wins,total_matches,trueskill_mu,trueskill_sigma
0,0,0,0,19.209512,36.644581,25.000000,8.333333
1,0,1,1,14.000000,24.000000,26.232905,4.854238
2,0,0,2,19.209512,36.644581,25.000000,8.333333
3,0,2,3,5.000000,8.000000,27.614505,6.550771
4,0,3,4,5.000000,16.000000,20.221006,5.961434


In [25]:
#Use as lookup table?
def static_stats(match_id, players, metric):
    stats = match_ratings[(match_ratings['match_id']==match_id)&(match_ratings['player_slot'].isin(players))]
    return stats[[metric]].mean()
    
#static_stats(0, [1, 2], 'total_wins')

In [31]:
aggStats['radiant_wins'] = playerTimes.apply(lambda x: 
    static_stats(x['match_id'], [x['P' + str(i)] for i in range(5)], 'total_wins'), axis = 1).values
aggStats['dire_wins'] = playerTimes.apply(lambda x: 
    static_stats(x['match_id'], [x['P' + str(i)] for i in range(128,133)], 'total_wins'), axis = 1).values

aggStats['radiant_matches'] = playerTimes.apply(lambda x: 
    static_stats(x['match_id'], [x['P' + str(i)] for i in range(5)], 'total_matches'), axis = 1).values
aggStats['dire_matches'] = playerTimes.apply(lambda x: 
    static_stats(x['match_id'], [x['P' + str(i)] for i in range(128,133)], 'total_matches'), axis = 1).values

aggStats['radiant_trueskill'] = playerTimes.apply(lambda x: 
    static_stats(x['match_id'], [x['P' + str(i)] for i in range(5)], 'trueskill_mu'), axis = 1).values
aggStats['dire_trueskill'] = playerTimes.apply(lambda x: 
    static_stats(x['match_id'], [x['P' + str(i)] for i in range(128,133)], 'trueskill_mu'), axis = 1).values

In [32]:
aggStats.head()

,fight_id,radiant_gold,dire_gold,radiant_xp,dire_xp,radiant_lh,dire_lh,radiant_wins,radiant_matches,radiant_trueskill,dire_wins,dire_matches,dire_trueskill,radiant_count,dire_count
0,0,765.0,939.25,814.5,833.25,5.75,10.00,10.802378,21.161145,24.767104,28.552378,44.911145,29.684593,3,4
1,1,1708.8,1953.20,1812.0,1789.00,15.40,21.60,12.483805,24.257832,24.813683,26.683805,43.257832,28.747675,4,5
2,2,4662.2,4717.00,4805.0,4578.75,48.60,58.75,12.483805,24.257832,24.813683,28.552378,44.911145,29.684593,4,4
3,3,7314.0,6465.20,7821.2,6307.60,72.80,78.60,12.483805,24.257832,24.813683,26.683805,43.257832,28.747675,4,5
4,4,9525.4,8266.40,10450.6,8163.20,94.00,93.20,12.483805,24.257832,24.813683,26.683805,43.257832,28.747675,4,5


In [28]:
def playerCount(players):
    return sum([1 for i in range(5) if players[i] > 0])

In [29]:
aggStats['radiant_count'] = playerTimes.apply(lambda x:
    playerCount([x['P' + str(i)] for i in range(5)]), axis = 1).values
aggStats['dire_count'] = playerTimes.apply(lambda x:
    playerCount([x['P' + str(i)] for i in range(128,133)]), axis = 1).values

In [30]:
aggStats.head()

,fight_id,radiant_gold,dire_gold,radiant_xp,dire_xp,radiant_lh,dire_lh,radiant_wins,radiant_matches,radiant_trueskill,dire_wins,dire_matches,dire_trueskill,radiant_count,dire_count
0,0,765.0,939.25,814.5,833.25,5.75,10.00,10.802378,21.161145,24.767104,28.552378,44.911145,29.684593,3,4
1,1,1708.8,1953.20,1812.0,1789.00,15.40,21.60,12.483805,24.257832,24.813683,26.683805,43.257832,28.747675,4,5
2,2,4662.2,4717.00,4805.0,4578.75,48.60,58.75,12.483805,24.257832,24.813683,28.552378,44.911145,29.684593,4,4
3,3,7314.0,6465.20,7821.2,6307.60,72.80,78.60,12.483805,24.257832,24.813683,26.683805,43.257832,28.747675,4,5
4,4,9525.4,8266.40,10450.6,8163.20,94.00,93.20,12.483805,24.257832,24.813683,26.683805,43.257832,28.747675,4,5


In [40]:
def getDifference(fight_id):
    kills = deathCount[fight_id][True] + deathCount[fight_id][False]
    difference = deathCount[fight_id][True] - deathCount[fight_id][False]
    KDD = difference / kills
    return KDD

In [45]:
aggStats['KDD'] = playerTimes['fight_id'].apply(getDifference).values

In [46]:
aggStats.head()

,fight_id,radiant_gold,dire_gold,radiant_xp,dire_xp,radiant_lh,dire_lh,radiant_wins,radiant_matches,radiant_trueskill,dire_wins,dire_matches,dire_trueskill,radiant_count,dire_count,KDD
0,0,765.0,939.25,814.5,833.25,5.75,10.00,10.802378,21.161145,24.767104,28.552378,44.911145,29.684593,3,4,-0.333333
1,1,1708.8,1953.20,1812.0,1789.00,15.40,21.60,12.483805,24.257832,24.813683,26.683805,43.257832,28.747675,4,5,-1.000000
2,2,4662.2,4717.00,4805.0,4578.75,48.60,58.75,12.483805,24.257832,24.813683,28.552378,44.911145,29.684593,4,4,-1.000000
3,3,7314.0,6465.20,7821.2,6307.60,72.80,78.60,12.483805,24.257832,24.813683,26.683805,43.257832,28.747675,4,5,-1.000000
4,4,9525.4,8266.40,10450.6,8163.20,94.00,93.20,12.483805,24.257832,24.813683,26.683805,43.257832,28.747675,4,5,-0.200000


In [ ]:
#naive regression, pre variable engineering
